# **Testing**

make sure that you load the dataset you want to test on and the .PTH file for the model that you want to use (The files are provided in the pretrained_model) folder. the images paths we used for testing for all the datasets are provided in the testing_dataset folder

make sure to go to the directory where the images are stored.

In [ ]:
cd /content/Data

The following cell will run the detector on all the images and store the prediction in the lists corresponding to each class

# **ResNet**

In [ ]:
import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image
import os

# Initialize lists to store paths for each class
CUTLER = []
DETR = []
CLIP = []
FTRCNN = []
MaskRCNN = []
OWL = []
RETNET = []
YOLO = []

# Set device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Load the fine-tuned model
model = models.resnet50(pretrained=False)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(class_names))  # Adjust the final layer
model.load_state_dict(torch.load('fine_tuned_resnet50.pth'))
model = model.to(device)
model.eval()

# Define the transformation to match the training preprocessing
data_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Function to predict the class of an image
def predict_image(image_path, model, transform, class_names):
    # Load and transform the image
    image = Image.open(image_path).convert('RGB')
    image = transform(image).unsqueeze(0)  # Add batch dimension

    # Move the image to the same device as the model
    image = image.to(device)

    # Perform inference
    with torch.no_grad():
        outputs = model(image)
        _, preds = torch.max(outputs, 1)

    # Get the predicted class name
    predicted_class = class_names[preds.item()]
    return predicted_class

# Directory containing new images
new_images_dir = '/content/Data'

# Iterate over images and predict their classes
for image_name in os.listdir(new_images_dir):
    if image_name.endswith(('.jpg', '.jpeg', '.png', '.bmp', '.tiff', '.gif')):  # Check if the file is an image
        image_path = os.path.join(new_images_dir, image_name)
        predicted_class = predict_image(image_path, model, data_transforms, class_names)
        print(f'Image: {image_name}, Predicted Class: {predicted_class}')

        # Append the path to the appropriate list based on the predicted class
        if predicted_class == 'CLIP':
            CLIP.append(image_path)
        elif predicted_class == 'CUTLER':
            CUTLER.append(image_path)
        elif predicted_class == 'DETR':
            DETR.append(image_path)
        elif predicted_class == 'FTRCNN':
            FTRCNN.append(image_path)
        elif predicted_class == 'MaskRCNN':
            MaskRCNN.append(image_path)
        elif predicted_class == 'OWL':
            OWL.append(image_path)
        elif predicted_class == 'RETNET':
            RETNET.append(image_path)
        elif predicted_class == 'YOLO':
            YOLO.append(image_path)


In [ ]:
print(len(OWL))
print(len(CLIP))
print(len(CUTLER))
print(len(DETR))
print(len(FTRCNN))
print(len(MaskRCNN))
print(len(RETNET))
print(len(YOLO))


In [ ]:
cd /content

In [ ]:
def write_list_to_txt(input_list, output_file_path):
    with open(output_file_path, 'w') as file:
        for item in input_list:
            file.write(str(item) + '\n')

# Example usage:
my_list =  OWL
output_file_path = '/content/OWL.txt'

write_list_to_txt(my_list, output_file_path)


# **ResNet(mocov2)**

In [ ]:
import torch
from torchvision import transforms
from PIL import Image
import os
from torchvision import models

CLIP=[]
CUTLER=[]
DETR=[]
FTRCNN=[]
MaskRCNN=[]
OWL=[]
RETNET=[]
YOLO=[]

# Define the path to your image folder
IMAGE_FOLDER = '/content/Data'

# Define transformations for input images
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize the image to 224x224
    transforms.ToTensor(),           # Convert the image to a PyTorch tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalize the image
])

# Load the fine-tuned ResNet50 MoCo v2 model
class ResNetMoCoClassification(torch.nn.Module):
    def __init__(self, num_classes):
        super(ResNetMoCoClassification, self).__init__()
        # Load ResNet model
        self.resnet = models.resnet50(pretrained=False)
        # Modify the final layer to match the number of classes
        self.resnet.fc = torch.nn.Linear(self.resnet.fc.in_features, num_classes)

    def forward(self, x):
        return self.resnet(x)

# Initialize the model
NUM_CLASSES = 8  # Change this to the number of classes in your dataset
model = ResNetMoCoClassification(NUM_CLASSES)

# Load the fine-tuned model weights
model.load_state_dict(torch.load('/content/resnet_moco_model.pth', map_location=torch.device('cpu')))

# Set the model to evaluation mode
model.eval()

# Loop through the image folder
for root, dirs, files in os.walk(IMAGE_FOLDER):
    for file in files:
        # Check if the file is an image
        if file.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):
            # Construct the image path
            image_path = os.path.join(root, file)

            # Load and preprocess the image
            image = Image.open(image_path)
            image_tensor = transform(image).unsqueeze(0)  # Add a batch dimension

            # Make predictions
            with torch.no_grad():
                output = model(image_tensor)

            # Get the predicted class
            _, predicted_class = torch.max(output, 1)
            predicted_class = predicted_class.item()

            # Replace 'class_names' with your actual class names
            class_names = ['CLIP','CUTLER','DETR','FTRCNN','MaskRCNN','OWL','RETNET','YOLO']

            predicted_class_name = class_names[predicted_class]

            print(f"Image: {file}, Predicted Class: {predicted_class_name}")

            if predicted_class_name == 'CLIP':
                CLIP.append(file)

            elif predicted_class_name == 'CUTLER':
                CUTLER.append(file)

            elif predicted_class_name == 'DETR':
                DETR.append(file)

            elif predicted_class_name == 'FTRCNN':
                FTRCNN.append(file)

            elif predicted_class_name == 'MaskRCNN':
                MaskRCNN.append(file)

            elif predicted_class_name == 'OWL':
                OWL.append(file)

            elif predicted_class_name == 'RETNET':
                RETNET.append(file)

            elif predicted_class_name == 'YOLO':
                YOLO.append(file)



In [ ]:
print(len(CLIP))
print(len(CUTLER))
print(len(DETR))
print(len(FTRCNN))
print(len(MaskRCNN))
print(len(OWL))
print(len(RETNET))
print(len(YOLO))

In [ ]:
cd /content

In [ ]:
def write_list_to_txt(input_list, output_file_path):
    with open(output_file_path, 'w') as file:
        for item in input_list:
            file.write(str(item) + '\n')

# Example usage:
my_list = YOLO
output_file_path = '/content/YOLO.txt'

write_list_to_txt(my_list, output_file_path)


# **ResNet(SwAV)**

In [ ]:
import torch
from torchvision import transforms
from PIL import Image
import os
from torchvision import models

# Lists to store predicted image paths
CLIP = []
CUTLER = []
DETR = []
FTRCNN = []
MaskRCNN = []
OWL = []
RETNET = []
YOLO = []

# Define the path to your image folder
IMAGE_FOLDER = '/content/Data'

# Define transformations for input images
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize the image to 224x224
    transforms.ToTensor(),           # Convert the image to a PyTorch tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalize the image
])

# Load the fine-tuned ResNet50 with SwAV weights
class ResNetSwAVClassification(torch.nn.Module):
    def __init__(self, num_classes):
        super(ResNetSwAVClassification, self).__init__()
        # Load ResNet model
        self.resnet = models.resnet50(pretrained=False)
        # Modify the final layer to match the number of classes
        self.resnet.fc = torch.nn.Linear(self.resnet.fc.in_features, num_classes)

    def forward(self, x):
        return self.resnet(x)

# Initialize the model
NUM_CLASSES = 8  # Change this to the number of classes in your dataset
model = ResNetSwAVClassification(NUM_CLASSES)

# Load the fine-tuned model weights
model.load_state_dict(torch.load('/content/resnet_swav_finetuned.pth', map_location=torch.device('cpu')))

# Set the model to evaluation mode
model.eval()

# Loop through the image folder
for root, dirs, files in os.walk(IMAGE_FOLDER):
    for file in files:
        # Check if the file is an image
        if file.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):
            # Construct the image path
            image_path = os.path.join(root, file)

            # Load and preprocess the image
            image = Image.open(image_path)
            image_tensor = transform(image).unsqueeze(0)  # Add a batch dimension

            # Make predictions
            with torch.no_grad():
                output = model(image_tensor)

            # Get the predicted class
            _, predicted_class = torch.max(output, 1)
            predicted_class = predicted_class.item()

            # Replace 'class_names' with your actual class names
            class_names = ['CLIP','CUTLER','DETR','FTRCNN','MaskRCNN','OWL','RETNET','YOLO']

            predicted_class_name = class_names[predicted_class]

            print(f"Image: {file}, Predicted Class: {predicted_class_name}")

            if predicted_class_name == 'CLIP':
                CLIP.append(file)
            elif predicted_class_name == 'CUTLER':
                CUTLER.append(file)
            elif predicted_class_name == 'DETR':
                DETR.append(file)
            elif predicted_class_name == 'FTRCNN':
                FTRCNN.append(file)
            elif predicted_class_name == 'MaskRCNN':
                MaskRCNN.append(file)
            elif predicted_class_name == 'OWL':
                OWL.append(file)
            elif predicted_class_name == 'RETNET':
                RETNET.append(file)
            elif predicted_class_name == 'YOLO':
                YOLO.append(file)

# Print the lists of predicted image paths for each class
print("CLIP:", CLIP)
print("CUTLER:", CUTLER)
print("DETR:", DETR)
print("FTRCNN:", FTRCNN)
print("MaskRCNN:", MaskRCNN)
print("OWL:", OWL)
print("RETNET:", RETNET)
print("YOLO:", YOLO)


In [ ]:
print(len(CLIP))
print(len(CUTLER))
print(len(DETR))
print(len(FTRCNN))
print(len(MaskRCNN))
print(len(OWL))
print(len(RETNET))
print(len(YOLO))

In [ ]:
def write_list_to_txt(input_list, output_file_path):
    with open(output_file_path, 'w') as file:
        for item in input_list:
            file.write(str(item) + '\n')

# Example usage:
my_list = YOLO
output_file_path = '/content/YOLO.txt'

write_list_to_txt(my_list, output_file_path)


# ResNet(**BT**)

In [ ]:
import torch
from torchvision import transforms
from PIL import Image
import os
from torchvision.models.resnet import Bottleneck, ResNet

# Lists to store predicted image paths
CLIP = []
CUTLER = []
DETR = []
FTRCNN = []
MaskRCNN = []
OWL = []
RETNET = []
YOLO = []

# Define the path to your image folder
IMAGE_FOLDER = '/content/Data'

# Define transformations for input images
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize the image to 224x224
    transforms.ToTensor(),          # Convert the image to a PyTorch tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalize the image
])

class ResNetTrunk(ResNet):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        del self.fc  # remove FC layer

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        return x

def get_pretrained_url(key):
    URL_PREFIX = "https://github.com/lunit-io/benchmark-ssl-pathology/releases/download/pretrained-weights"
    model_zoo_registry = {
        "BT": "bt_rn50_ep200.torch",
    }
    pretrained_url = f"{URL_PREFIX}/{model_zoo_registry.get(key)}"
    return pretrained_url

def resnet50(pretrained, progress, key, **kwargs):
    model = ResNetTrunk(Bottleneck, [3, 4, 6, 3], **kwargs)
    if pretrained:
        pretrained_url = get_pretrained_url(key)
        verbose = model.load_state_dict(
            torch.hub.load_state_dict_from_url(pretrained_url, progress=progress),
            strict=False  # allow missing keys
        )
        print(verbose)
    return model

# Load the fine-tuned ResNet50 with BT weights
class ResNetBTClassification(torch.nn.Module):
    def __init__(self, num_classes):
        super(ResNetBTClassification, self).__init__()
        # Load ResNet model with BT pre-trained weights
        self.trunk = resnet50(pretrained=True, progress=False, key="BT")
        # Modify the final layer to match the number of classes
        self.fc = torch.nn.Linear(2048, num_classes)  # Assuming the output from trunk has 2048 features

    def forward(self, x):
        x = self.trunk(x)
        x = torch.nn.functional.adaptive_avg_pool2d(x, (1, 1))
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

# Initialize the model
NUM_CLASSES = 8  # Change this to the number of classes in your dataset
model = ResNetBTClassification(NUM_CLASSES)

# Load the fine-tuned model weights
model.load_state_dict(torch.load('/content/resnet_bt_finetuned.pth', map_location=torch.device('cpu')), strict=False)

# Set the model to evaluation mode
model.eval()

# Loop through the image folder
for root, dirs, files in os.walk(IMAGE_FOLDER):
    for file in files:
        # Check if the file is an image
        if file.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):
            # Construct the image path
            image_path = os.path.join(root, file)

            # Load and preprocess the image
            image = Image.open(image_path)
            image_tensor = transform(image).unsqueeze(0)  # Add a batch dimension

            # Make predictions
            with torch.no_grad():
                output = model(image_tensor)

            # Get the predicted class
            _, predicted_class = torch.max(output, 1)
            predicted_class = predicted_class.item()

            # Replace 'class_names' with your actual class names
            class_names = ['CLIP','CUTLER','DETR','FTRCNN','MaskRCNN','OWL','RETNET','YOLO']

            predicted_class_name = class_names[predicted_class]

            print(f"Image: {file}, Predicted Class: {predicted_class_name}")

            if predicted_class_name == 'CLIP':
                CLIP.append(file)
            elif predicted_class_name == 'CUTLER':
                CUTLER.append(file)
            elif predicted_class_name == 'DETR':
                DETR.append(file)
            elif predicted_class_name == 'FTRCNN':
                FTRCNN.append(file)
            elif predicted_class_name == 'MaskRCNN':
                MaskRCNN.append(file)
            elif predicted_class_name == 'OWL':
                OWL.append(file)
            elif predicted_class_name == 'RETNET':
                RETNET.append(file)
            elif predicted_class_name == 'YOLO':
                YOLO.append(file)

# Print the lists of predicted image paths for each class
print("CLIP:", CLIP)
print("CUTLER:", CUTLER)
print("DETR:", DETR)
print("FTRCNN:", FTRCNN)
print("MaskRCNN:", MaskRCNN)
print("OWL:", OWL)
print("RETNET:", RETNET)
print("YOLO:", YOLO)


In [ ]:
print(len(CLIP))
print(len(CUTLER))
print(len(DETR))
print(len(FTRCNN))
print(len(MaskRCNN))
print(len(OWL))
print(len(RETNET))
print(len(YOLO))

In [ ]:
def write_list_to_txt(input_list, output_file_path):
    with open(output_file_path, 'w') as file:
        for item in input_list:
            file.write(str(item) + '\n')

# Example usage:
my_list = YOLO
output_file_path = '/content/YOLO.txt'

write_list_to_txt(my_list, output_file_path)
